In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

    
    
    
def Belief(State,p,r):
   
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;
            
        
def WhittleIndex(State,p,r):
    WhittleIndex = 0.;

    
    if State[0]== 0 and State[1]>0:
        l= State[1];
        x= Belief([0,l],p,r);
        y=Belief([0,l+1],p,r);
        WhittleIndex =((x-y)*(l+1)+y)/(1-p+(x-y)*l+y); 
    elif State[0] == 1:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
    
    else:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
            
    return WhittleIndex;

def update(S, C,alpha,p,r,N):
        #the update process of each timestep.
        
        #S is the list of states of all channels.
        W = [0.]*N;
        Y = [0.]*N;
        Action = [0]*N;
        Threshold = 0.;
        Seed = np.random.rand(N);
        Reward = 0;
        
       #Get the Threshold. 
        for i in range(N):
            Y[i]= WhittleIndex(S[i],p,r);
        
        Y.sort();
        Threshold = Y[int((1-alpha)*N)];
        
        
        
       #Get the Action vector.
        for i in range(N):
            W[i]= WhittleIndex(S[i],p,r);
    
        h=0;
        for i in range(N):
            if W[i]>= Threshold:
                Action[i]=1;
                h=h+1;
            if h >= alpha*N:
                break;
            
       #The update process.
       
        for i in range(N):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<p)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<r)==True);
        
        
        return Reward;
    


In [2]:
print(update([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0.4,0.6,0.3,10));

1


In [3]:
def onetrial(S,C,timesteps,alpha,p,r,N):
    Reward = 0;
    for i in range(timesteps):
        Reward += update(S,C,alpha,p,r,N);
    return Reward;

In [9]:
print(onetrial([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,1],100,0.4,0.6,0.2,10));

193


In [10]:

def TransitionMatrix(t,rho,p,r):
    M = np.zeros((t+2,t+2));
    for i in range (t-1):
        M[i,i+1]=1;
    M[t-1,0]= rho*(1-Belief([0,t],p,r));
    M[t-1,t]= 1-rho;
    M[t-1,t+1]= rho*Belief([0,t],p,r);
    M[t,0]= 1-Belief([0,t+1],p,r);
    M[t,t+1]= Belief([0,t+1],p,r);
    M[t+1,0]= 1-Belief([1,1],p,r);
    M[t+1,t+1]= Belief([1,1],p,r);
    return M;




In [11]:
def steady_state_prop(p):
    dim = p.shape[0]
    q = (p-np.eye(dim))
    ones = np.ones(dim)
    q = np.c_[q,ones]
    QTQ = np.dot(q, q.T)
    bQT = np.ones(dim)
    return np.linalg.solve(QTQ,bQT)

In [12]:
def find_threshold(alpha,p,r):
    threshold=0;
    while threshold >=0:
        steadystate= steady_state_prop(TransitionMatrix(threshold,0.0001,p,r));
        active= steadystate[threshold-1]*0.0001+steadystate[threshold]+steadystate[threshold+1];
        if active< alpha:
            break
        threshold += 1;
    return threshold;


In [13]:
def find_rho(alpha,p,r):
    t= find_threshold(alpha,p,r);
    h=1;
    ind=1;
    
    for i in range(10000):
        M = TransitionMatrix(t,i*0.0001,p,r);
        steadystate= steady_state_prop(M);
        active= steadystate[t-1]*i*0.0001+steadystate[t]+steadystate[t+1];
        if abs(active-alpha)<h:
            h= abs(active-alpha);
            ind= i;
            
    return ind*0.0001;

In [14]:
def initial_belief(N,alpha,p,r):
    S = [[0,0]]*N;
    t = find_threshold(alpha,p,r);
    rho = find_rho(alpha,p,r);
    steadystate= steady_state_prop(TransitionMatrix(t,rho,p,r));
    x= int(N*steadystate[0]);
    for i in range(t):
        for j in range(x):
            S[j+i*x]= [0,i+1];
    y= int(steadystate[t]*N);
    for i in range(x*t,x*t+y):
        S[i]= [0,t+1];
    for i in range(x*t+y,N):
        S[i]= [1,1];
    return S;
        
        
    
    
    

In [16]:
print(initial_belief(10,0.4,0.7,0.4));

[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]


In [17]:
import copy;

def mc(ntrials,timesteps,alpha,p,r,N):
    equilibrium = r/(1-p+r);
    
    S= initial_belief(N,alpha,p,r);
    Reward=0;
    C= [0]*N;
     
    for i in range(ntrials):
        SS= copy.deepcopy(S);
        CC= C;
        Seed = np.random.rand(N);
        for j in range(N):
            CC[j]= int((Seed[j]< equilibrium) == True);
            
    
        one_time_reward = onetrial(SS,CC,timesteps,alpha,p,r,N);
        Reward = Reward+one_time_reward;
       
        
    lowerbound = 100*Reward/(timesteps*ntrials*N);
    
    print(lowerbound);


In [20]:
mc(50,1000,0.4,0.7,0.4,10);

26.0504


In [89]:
mc(500,1000,0.4,0.7,0.4,500);

26.1199076


In [90]:
mc(500,1000,0.4,0.7,0.4,1000);

26.1561834


In [92]:
mc(500,1000,0.4,0.7,0.4,1500);

26.16631093333333


In [93]:
mc(500,1000,0.4,0.7,0.4,2000);

26.1738241


In [94]:
mc(500,1000,0.4,0.7,0.4,2500);

26.17523744


In [16]:
def lb(N):
    lb= mc(500,1000,0.4,0.7,0.4,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 26.213

25.99672
25.96139
25.95826
25.983035
25.989608
26.013612
26.053762


In [18]:
def lb(N):
    lb= mc(500,1000,0.4,0.7,0.1,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 16.360

15.11204
15.0309
15.07132
15.151135
15.225636
15.4682
15.59623


In [91]:
mc(500,1000,0.4,0.7,0.1,1000);

15.70469


In [95]:
mc(500,1000,0.4,0.7,0.1,2000);

15.7376649


In [19]:
mc(500,1000,0.4,0.7,0.1,2500);

15.74483408


In [21]:
def lb(N):
    lb= mc(500,1000,0.4,0.8,0.2,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 28.191

27.47604
27.46874
27.516693333333333
27.534015
27.566244
27.623274
27.701735


In [22]:
mc(500,1000,0.4,0.8,0.2,1000);

27.8166748


In [23]:
mc(500,1000,0.4,0.8,0.2,2000);

27.8465467


In [24]:
mc(500,1000,0.4,0.8,0.2,2500);

27.85617248


In [39]:
def lb(N):
    lb= mc(500,1000,0.4,0.85,0.25,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];


#The upperbound is 32.181

31.85306
31.87321
31.89122
31.88525
31.901928
31.92582
31.949833


In [40]:
mc(500,1000,0.4,0.85,0.25,1000);

32.0471892


In [12]:
mc(500,1000,0.4,0.85,0.25,2000);

32.0842939


In [ ]:
mc(500,1000,0.4,0.85,0.25,2500);

In [14]:
def lb(N):
    lb= mc(500,1000,0.4,0.8,0.55,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];


#The upperbound is 31.428

31.3622
31.3484
31.34545333333333
31.354545
31.336424
31.346258
31.358815


In [ ]:
mc(500,1000,0.4,0.8,0.55,1000);

In [ ]:
mc(500,1000,0.4,0.8,0.55,2000；

In [ ]:
mc(500,1000,0.4,0.8,0.55,2500);

In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.9,0.6,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];


#The upperbound is 35.821

35.81112
35.80998
35.81238666666667
35.79548
35.793384
35.796726666666665
35.801634285714286
35.790925
35.78768888888889
35.786096
35.79165
35.790741


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.4,0.2,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];


#The upperbound is 11.704

11.27528
11.31232
11.341306666666666
11.34234
11.357856
11.387486666666666
11.39444
11.39734
11.39568
11.429376
11.475024
11.537278
